In [1]:
import numpy as np
import matplotlib_inline
import matplotlib.pyplot as plt
import matplotlib
import pandas
import sklearn
import sklearn.datasets

Task 1: Implementing a Logistic Regression Model

In [2]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [3]:
def binary_cross_entropy(y_true, y_pred):
    return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

In [5]:
def logistic_regression(X, y, learning_rate=0.01, num_epochs=100, threshold=0.5, max_iter=1000):
    # Initialize weights and bias
    w = np.zeros(X.shape[1])
    b = 0

    stopping = False
    J_running = 0
    J_running_prev = 0
    iteration = 0

    while not stopping:
        xi, yi = np.random.choice(len(y), 1, replace=False), np.random.choice(len(y), 1, replace=False)

        z = np.dot(w, X[xi]) + b
        y_hat = sigmoid(z)

        J_current = binary_cross_entropy(y[yi], y_hat)

        dw = (y_hat - y[yi]) * X[xi]
        db = y_hat - y[yi]

        w -= learning_rate * dw
        b -= learning_rate * db

        iteration += 1
        J_running += J_current

        if iteration > max_iter:
            stopping = True

        if (iteration % num_epochs == 0) and (np.abs(J_running - J_running_prev) < threshold):
            stopping = True
            J_running_prev = J_running
            J_running = 0

    return w, b